In [9]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import os

In [10]:
aws_access_key = os.environ["AWS_ACCESS_KEY"]
aws_secret_key = os.environ["AWS_SECRET_KEY"]
aws_region = os.environ["AWS_REGION"]
warehouse_location = os.environ["WAREHOUSE_LOCATION"]
metastore_uri = os.environ["METASTORE_URI"]

spark = SparkSession.builder.appName("Warehouse")\
    .master("spark://spark-master:7077") \
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider") \
    .config("spark.hadoop.fs.s3a.access.key", aws_access_key) \
    .config("spark.hadoop.fs.s3a.secret.key", aws_secret_key) \
    .config("spark.sql.catalogImplementation", "hive") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("spark.sql.hive.metastore.uris", metastore_uri) \
    .config("hive.metastore.uris", metastore_uri) \
    .config("hive.metastore.warehouse.dir", warehouse_location) \
    .config("hive.hadoop.fs.s3a.access.key", aws_access_key) \
    .config("hive.hadoop.fs.s3a.secret.key", aws_secret_key) \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.driver.memory", "5G") \
    .config("spark.memory.offHeap.size","16g") \
    .config("spark.memory.offHeap.enabled", True) \
    .enableHiveSupport() \
    .getOrCreate()

In [11]:
spark.sql("show tables in craigslist_vehicles_bronze").show()

+--------------------+-------------------+-----------+
|           namespace|          tableName|isTemporary|
+--------------------+-------------------+-----------+
|craigslist_vehicl...|craigslist_vehicles|      false|
+--------------------+-------------------+-----------+



In [12]:
year_id = spark.sql("""
    select distinct year, COLLECT_LIST(region), state, count(*) as count
    from craigslist_vehicles_bronze.craigslist_vehicles
    group by year, state
""")
year_id.show(truncate=False)
year_id.write.mode("overwrite").saveAsTable("facts.year_id")

+----------+--------------------------------------------------------------------------------+-----+-----+
|year      |collect_list(region)                                                            |state|count|
+----------+--------------------------------------------------------------------------------+-----+-----+
|1900-01-01|[muncie / anderson]                                                             |in   |1    |
|1901-01-01|[phoenix]                                                                       |az   |1    |
|1903-01-01|[birmingham]                                                                    |al   |1    |
|1903-01-01|[norfolk / hampton roads]                                                       |va   |1    |
|1909-01-01|[tyler / east TX]                                                               |tx   |1    |
|1920-01-01|[bend]                                                                          |or   |1    |
|1923-01-01|[hartford]                        

In [13]:
year_id_price = spark.sql("""
    select year, sum(price) as price, COLLECT_LIST(region), state
    from craigslist_vehicles_bronze.craigslist_vehicles
    where price is not null and price > 0
    group by year, state
""")
year_id_price.show(truncate=False)
year_id_price.write.mode("overwrite").saveAsTable("facts.year_id_price")

+----------+-------+--------------------------------------------------------------------------------+------------------------------------------------------------------------------+
|year      |price  |collect_list(region)                                                            |state                                                                         |
+----------+-------+--------------------------------------------------------------------------------+------------------------------------------------------------------------------+
|1900-01-01|75.0   |[muncie / anderson]                                                             |in                                                                            |
|1901-01-01|999.0  |[phoenix]                                                                       |az                                                                            |
|1923-01-01|18500.0|[hartford]                                                                 

In [14]:
manufacturer_year = spark.sql("""
    select distinct manufacturer, year, COLLECT_LIST(region), count(*) as count, state
    from craigslist_vehicles_bronze.craigslist_vehicles
    where manufacturer is not null and year is not null
    group by manufacturer, year, state
""")
manufacturer_year.show(truncate=False)
manufacturer_year.write.mode("overwrite").saveAsTable("facts.manufacturer_year")

+------------+----------+--------------------------------------------------------------------------------------------------------------------+-----+-----+
|manufacturer|year      |collect_list(region)                                                                                                |count|state|
+------------+----------+--------------------------------------------------------------------------------------------------------------------+-----+-----+
|acura       |1988-01-01|[yakima]                                                                                                            |1    |wa   |
|acura       |1996-01-01|[south florida]                                                                                                     |1    |NULL |
|acura       |1998-01-01|[modesto, sacramento, SF bay area]                                                                                  |3    |ca   |
|acura       |2001-01-01|[albany]                                     

In [15]:
region_year = spark.sql("""
    select distinct id, COLLECT_LIST(region), year, state
    from craigslist_vehicles_bronze.craigslist_vehicles
    where region is not null and year is not null
    group by id, year, state
""")
region_year.show(truncate=False)
region_year.write.mode("overwrite").saveAsTable("facts.region_year")

+----------+----------------------------+----------+-----+
|id        |collect_list(region)        |year      |state|
+----------+----------------------------+----------+-----+
|7301585649|[cleveland]                 |1958-01-01|oh   |
|7301590460|[ventura county]            |2010-01-01|ca   |
|7301591111|[williamsport]              |2018-01-01|pa   |
|7301591170|[western massachusetts]     |2011-01-01|ma   |
|7301591660|[memphis]                   |2008-01-01|tn   |
|7301594230|[baton rouge]               |2019-01-01|la   |
|7301594252|[tallahassee]               |2018-01-01|fl   |
|7301594276|[redding]                   |2020-01-01|ca   |
|7301594875|[santa barbara]             |2016-01-01|ca   |
|7301595562|[pueblo]                    |2018-01-01|co   |
|7301597073|[merced]                    |2017-01-01|ca   |
|7301597103|[merced]                    |2020-01-01|ca   |
|7301597908|[akron / canton]            |2021-01-01|oh   |
|7301597969|[merced]                    |2019-01-01|ca  

In [16]:
spark.stop()